In [ ]:
# default_exp srag_data

# SRAG Data

> Get SRAG data from opendatasus

In [ ]:
#hide
from nbdev.showdoc import *
import os.path
import pandas as pd
import ssl
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [ ]:
#export
def get_last_bd_srag_csv_url(year=2021):
    
    available_years = (2020,2021)
    if year not in available_years:
        print('year not valid. Available years:',available_years)
        return
    
    # Se nao achar, retorna última url encontrada
    srag_url = f'https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/{year}/INFLUD-29-03-2021.csv'
    
    context = ssl._create_unverified_context() # To aviod ssl error
    html_page = urlopen(f'https://opendatasus.saude.gov.br/dataset/bd-srag-{year}', context=context)
    soup = BeautifulSoup(html_page, features="lxml")
    for link in soup.findAll('a'):
        url = link.get('href')
        (filename, ext) = os.path.splitext(url)
        if ext.lower() == '.csv':
            srag_url = url
            print(f'\nNew csv file url: <{srag_url}>')
    
    return srag_url



def get_srag_data(year=2021,update=False,treat=True,selected_columns='BASIC',aditional_columns=[]):
    
    sep = ';'
    quotechar = '"'
    
    fname = f'data/SRAG/INFLUD{year}.csv'
    if os.path.isfile(fname) and not update:
        print(f'\nReading OpenDataSus from local file <{fname}>. If you prefer to download last version, set "update=True".\n')
        df = pd.read_csv(fname,dtype=object)
    else:
        url = get_last_bd_srag_csv_url(year)
        print(f'\nDownloading from <{url}> ... ', end='')
        df = pd.read_csv(url,sep=sep,quotechar=quotechar,dtype=object)
        df.to_csv(fname,index=False)
        print('complete!\n')
        
    if treat:
        df = treat_srag_data(df,selected_columns,aditional_columns)
    
    return df

def get_cities_states_dictionaries():
    ''' Returns 2 dictionaries: 
    1. cities_dict - city code (6 dig): city name
    2. states_dict - state code (2 dig): state name
    '''
    fname = 'data/IBGE/RELATORIO_DTB_BRASIL_MUNICIPIO.ods'
    df = pd.read_excel(fname,dtype=object)
    df['cod_municipio'] = df['Código Município Completo'].str[:6]
    cities_dict = df.set_index('cod_municipio')['Nome_Município'].to_dict()
    states_dict = df[['UF','Nome_UF']].groupby('UF').first()['Nome_UF'].to_dict()
    return cities_dict, states_dict

def treat_srag_data(df,selected_columns='BASIC',aditional_columns=[]):
    "Select columns, set types and replace values."
    df = df.copy()
    date_cols = ['DT_SIN_PRI','DT_EVOLUCA','DT_NASC','DT_ENTUTI']
    cities_cols = ['CO_MUN_RES','CO_MU_INTE','CO_MUN_NOT']
#     str_cols = cities_cols
    
    if selected_columns != 'ALL':
        basic_cols = date_cols + cities_cols
        basic_cols += ['SEM_PRI', 'EVOLUCAO', 'CLASSI_FIN','CLASSI_OUT',                
                      'NU_IDADE_N','CS_RACA', 'CS_ESCOL_N', 'CS_SEXO',
#                       'ID_MN_RESI','ID_MN_ITE','ID_MUNICIP',
                      'UTI', 'SUPORT_VEN']
    
        if selected_columns == 'BASIC':
            cols = basic_cols
        else:
            cols = basic_cols + list(aditional_columns)
        df = df[cols]
        
    df_cols = df.columns
    numeric_cols = ['SEM_PRI','NU_IDADE_N']
#     numeric_cols = list(set(df_cols) - set(date_cols) - set(str_cols))
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce',dayfirst=True)

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    
    cities_dict,states_dict = get_cities_states_dictionaries()
    regions_dict = {'1':'Norte',
                    '2':'Nordeste',
                    '3':'Sudeste',
                    '4':'Sul',
                    '5':'Centro-Oeste' }
    
    for col in cities_cols:
#         city_name_col = col[3:]
        sufix_index = col.index('_',3)
        city_name_col = 'MUN' + col[sufix_index:]
        state_name_col = 'UF' + col[sufix_index:]
        region_name_col = 'REGIAO' + col[sufix_index:]
        df[city_name_col] = df[col].apply(lambda x: cities_dict.get(x,'n.d.'))
        df[state_name_col] = df[col].str[:2].apply(lambda x: states_dict.get(x,'n.d.'))
        df[region_name_col] = df[col].str[0].apply(lambda x: regions_dict.get(x,'n.d.'))
        
    evolucao_dict = {'1':'cura',
                     '2':'obito',
                     '3':'obito_outras_causas',
                     '9':'ignorado' }
    classi_fin_dict = {'1':'cura',
                     '2':'obito',
                     '3':'obito_outras_causas',
                     '9':'ignorado' }

    df['EVOLUCAO'] = df['EVOLUCAO'].apply(lambda x: evolucao_dict.get(x,'n.d.'))
    df['CLASSI_FIN'] = df['CLASSI_FIN'].apply(lambda x: classi_fin_dict.get(x,'n.d.'))
    dict_cols = ['EVOLUCAO','CLASSI_FIN']
    
    other_cols = list(set(df_cols) - set(date_cols) - set(numeric_cols) - set(cities_cols) - set(dict_cols))
    df[other_cols] = df[other_cols].fillna('n.d.')
#     for col in str_cols:
#         df[col] = df[col].fillna('n.d.')
    

        
    return df

In [ ]:
df_srag = get_srag_data(treat=False)


New csv file url: <https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2021/INFLUD21-29-03-2021.csv>




In [ ]:
df_srag.shape[0]

421368

In [ ]:
assert df_srag.shape[0]>=421368
assert df_srag.shape[1]==154

In [ ]:
df = treat_srag_data(df_srag)

In [ ]:
assert df.shape[0]==df_srag.shape[0]
assert df.shape[1]==26